#Бенчмарк. Определение остановок автобусов.

Ночь Анализа Данных Яндекса, 2015

Подключим необходимые библиотеки:

In [1]:
%pylab inline

import numpy as np
import pandas as pd

from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier

Populating the interactive namespace from numpy and matplotlib


Считаем файлы с обучающей и тестовой выборкой, а также файл с остановками.

In [2]:
dtrain = pd.read_csv('gps_train.tsv', sep = '\t', names = ['datetime', 'id', 'type', 'hash', 'latitude', 'longitude'])
dtest = pd.read_csv('gps_test.tsv', sep = '\t', names = ['datetime', 'id', 'type', 'hash', 'latitude', 'longitude'])

dstops = pd.read_csv('stops_train.tsv', sep = '\t', names = ['latitude', 'longitude'])

Создадим массивы с координатами точек.

In [3]:
coordtrain = np.array(dtrain.ix[:, 4:])
coordtest = np.array(dtest.ix[:, 4:])

coordstops = np.array(dstops.ix[:, :])

Создадим обучающую выборку.
Для этого пометим как остановкии те точки маршрутов, которые наиболее близки к реальным остановкам.

In [4]:
nbrs = NearestNeighbors(n_neighbors = 1, algorithm = 'ball_tree').fit(coordtrain)
selected = nbrs.kneighbors(coordstops)[1][:, 0]

ytrain = np.zeros(len(dtrain))
ytrain[selected] = 1

В качестве признаков возьмем расстояния до 5 ближайших точек из обучающей и тестовой выборки соответственно.

In [5]:
nbrs = NearestNeighbors(n_neighbors = 5, algorithm = 'ball_tree').fit(coordtrain)
Xtrain = nbrs.kneighbors(coordtrain)[0]

nbrs = NearestNeighbors(n_neighbors = 5, algorithm = 'ball_tree').fit(coordtest)
Xtest = nbrs.kneighbors(coordtest)[0]

Обучим модель случайного леса на наших данных.

In [6]:
model = RandomForestClassifier(n_estimators = 10, random_state = 0, n_jobs = -1)
model.fit(Xtrain, ytrain)

result = model.predict_proba(Xtest)[:, 1]

В качестве результата возьмем 3000 наиболее вероятных точек.
Выведем результат в файл.

In [7]:
np.savetxt('output.txt', coordtest[argsort(result)[-3000:]], header = '3000', comments = '')